In [24]:
#import libraries
import random
import sqlite3
from sqlite3 import Error

#define bestseller list
bestsellerslist = ['Windshield Sun Shade','Car Vacuum','Car Trash Can','Wiper Blade','Window Shade','Headrest Hanger','Windshield Sun Shade plus Steering Wheel Sun Shade','Car Seat Covers','2-Piece Sun Shade','Microfiber Cleaning Cloth','Garbage Guard','Gel Seat Cushion','Trunk Organizers and Storage','Rubber Floor Mats','RV Sewer Hose','Office Chair Seat Cushion','Cabin Filter with Activated Carbon','LED Car Strip Lights','Cleaning Gel','RV Toilet Treatment','Cleaning Putty','Bumper Retainer Clips','Bike Phone Holder','10-Foot RV Sewer Hose Extension Kit','Car Wax Polish Spray','Interior Cleaner','Diamond Steering Wheel Cover','Portable Car Vacuum','180 Battery Organizer','Heavy Duty Threadlocker']
len(shoppinglist)

30

In [29]:
#generate 20 transactions
def generate_transactions(inputlist):
    index = 0
    transactions = []
    while index < 20:
        
        #generate random number of items in each shopping transaction
        transnum = random.randrange(1,20,1)

        #generate transactions using the transnum
        transaction = random.sample(inputlist,transnum)

        #sort alphabetically and append to transactions list
        transaction.sort()
        transactions.append(transaction)

        index +=1
    
    return transactions

#show transactions
transactions = generate_transactions(bestsellerslist)
for listitem in range(len(transactions)):
    print(transactions[listitem])


['Interior Cleaner', 'Portable Car Vacuum', 'RV Sewer Hose']
['Car Seat Covers', 'Car Trash Can', 'Cleaning Putty', 'Gel Seat Cushion', 'Heavy Duty Threadlocker', 'LED Car Strip Lights', 'Windshield Sun Shade']
['Car Trash Can', 'Car Wax Polish Spray', 'Cleaning Gel', 'Diamond Steering Wheel Cover', 'Garbage Guard', 'Gel Seat Cushion', 'Headrest Hanger', 'Heavy Duty Threadlocker', 'Interior Cleaner', 'Microfiber Cleaning Cloth', 'RV Toilet Treatment', 'Window Shade', 'Windshield Sun Shade plus Steering Wheel Sun Shade', 'Wiper Blade']
['10-Foot RV Sewer Hose Extension Kit', 'Bike Phone Holder', 'Bumper Retainer Clips', 'Car Seat Covers', 'Cleaning Gel', 'Diamond Steering Wheel Cover', 'Garbage Guard', 'Heavy Duty Threadlocker', 'Interior Cleaner', 'LED Car Strip Lights', 'Microfiber Cleaning Cloth', 'Office Chair Seat Cushion', 'RV Sewer Hose', 'Rubber Floor Mats', 'Trunk Organizers and Storage', 'Window Shade', 'Windshield Sun Shade plus Steering Wheel Sun Shade', 'Wiper Blade']
['10-

In [424]:
#create database(s) and relevant tables

def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    return conn

def create_table(conn,name):
    """ create table from the create_table_statement"""
    try:
        c = conn.cursor()
        c.execute("""CREATE TABLE IF NOT EXISTS %s (item_id text PRIMARY KEY)""" %(name))
        print('Table created')
        conn.commit()
        conn.close()
    except Error as e:
        print(e)
    

def populate_table(conn,tablename,itemname):
    """populate table from populate_statement"""
    try:
        c = conn.cursor()
        c.execute("""INSERT INTO %s (item_id) VALUES (?)""" %(tablename),[itemname])
        print('Table populated')
        conn.commit()
        conn.close()
    except Error as e:
        print(e)


def query(conn,statement):
    try:
        c = conn.cursor()
        c.execute(statement)
        conn.commit()
        row = c.fetchall()
        print(row)
        conn.close()
    except Error as e:
        print(e)
        
def main():
    database = r"C:\Users\Nick\CS634_Data_Mining\auto_best_sellers_1.db"
    
    index = 0
    #create the 20 transaction tables
    for index in range(len(transactions)):
        #create database connection
        conn = create_connection(database)
        if conn is not None:
            table_name = 'transaction_'+str(index)
            create_table(conn,table_name)
            #populate each table with the items in each respective transaction
            for item in transactions[index]:
                item_name = item
                conn = create_connection(database)
                populate_table(conn,table_name,item_name)
        else:
            print("Cannot connect to database")
            
#test
if __name__ == '__main__':
    #create_connection(r"C:\Users\Nick\CS634_Data_Mining\auto_best_sellers_1.db")
    #main()
    statement = "SELECT * FROM transaction_1 ORDER BY item_id ASC"
    #statement = "UPDATE transaction_15 SET item_id = 'Cleaning Putty' WHERE item_id='10-Foot RV Sewer Hose Extension Kit'"
    #statement = "UPDATE transaction_5 SET item_id = 'Cleaning Gel' WHERE item_id='Car Seat Covers'"
    #statement = "UPDATE transaction_1 SET item_id = 'Car Seat Covers' WHERE item_id='Cabin Filter with Activated Carbon'"
    #statement = "INSERT INTO transaction_14(item_id) VALUES ('RV Toilet Treatment')"
    statement = "INSERT INTO transaction_7(item_id) VALUES ('Garbage Guard')"
    #statement = "DELETE FROM transaction_6 WHERE item_id = 'RV Toilet Treatment'"
    query(create_connection(r"C:\Users\Nick\CS634_Data_Mining\auto_best_sellers_5.db"),statement)


2.6.0
[]


In [3]:
## Initialize the algorithm

In [425]:
def generateGroups(input_set,size):
    input_set = list(input_set)
    if size == 0:
        return [[]]
    new_set = []
    for index in range(0,len(input_set)):
        item = input_set[index]
        remaining = input_set[index + 1:]
        for j in generateGroups(remaining,size-1):
            new_set.append([item]+list(j))
    return new_set

def unionSets(input_1,input_2):
    return set(input_1).union(set(input_2))

def getItems(conn,transnum):
    try:
        items_list = []
        c = conn.cursor()
        #query the data and result will be in lexographical order
        c.execute("SELECT item_id FROM %s ORDER BY item_id ASC"%('transaction_'+str(transnum)))
        transaction = c.fetchall()
        for item in transaction:
            items_list.append(item[0])
        return items_list
        conn.close()
    except Error as e:
        print(e)

def getTransactions(conn,translength):
        transactions = []
        for i in range(translength):
            transaction = getItems(conn,i)
            transactions.append(transaction)
        return transactions

def scanTransactions(itemset,transactions):
    count = 0
    for i in range(len(transactions)):
        if set(itemset).issubset(transactions[i]):
            count +=1
    return count   

def getUnions(set1,n):
    sets = []
    for item in set1:
        if type(item) == type(''):
            item = [item]
            for i in set1:
                if type(i) == type(''):
                    i = [i]
                    if item != i:
                        u = set(item).union(set(i))
                        if u not in sets and len(u) == n:
                            sets.append(u)
        else:
            itemset = list(item)
            #print(set(itemset))
            for j in set1:
                if type(j) != type((1,2)):
                    u = set(itemset).union({j})
                    if u not in sets and len(u) == n:
                        sets.append(u)
                else:
                    itemset2 = set(list(j))
                    u = itemset2.union(itemset)
                    if u not in sets and len(u) == n:
                        sets.append(u)
                      
    return sets

def apriori(set_list,min_support):
    #initialize
    discarded = {}
    C = {}
    L = {}
    stop = False
    n = 1
    while stop == False:
        if n <= 1:
            for transaction in range(len(set_list)):
                for item in set_list[transaction]:
                    #prevent duplicate scans of same subset
                    if item not in C:
                        #prevent scans of already discarded items and previously non-frequent sets
                        if item not in discarded:
                            sup_count = scanTransactions({item},set_list)
                            C[item] = sup_count

            #print first scan results
            print('\nIteration {}'.format(n))
            print('C1 candidate set: \n')
            
            for i in C:
                print(i,':',C[i])
            #create L with items that meet minimum support requirement
            for item in C:
                sup_calc = C[item] / len(set_list)
                if item not in discarded:
                    if sup_calc >= min_support:
                        L[item] = C[item]
                    else:
                        discarded[item] = sup_calc
                    
            print('\nL1 set: \n')
            for i in L:
                print(i,':',L[i])
            print('#'*75)
            
            L_n = L #helps to generate sets for second iteration 
            n += 1
            if L is None:
                stop = True
                break
        elif n > 1:
            C_n = {}
            #union of the sets
            sets = []
            L_n = list(L_n)
            sets = getUnions(L_n,n)
            
            #save last iteration's results to a variable
            L_n_minus_1 = L_n
            
            #reinitialize L_n for the following iteration
            L_n = {}
            
            for i in range(len(sets)):
                support_count = scanTransactions(sets[i],set_list)
                C_n[tuple(sets[i])] = support_count
            if C_n == {}:
                stop = True
                break
            
            #print candidate set
            print('\nIteration {}\n'.format(n))
            print('C{} candidate set: \n'.format(n))
            for i in C_n:
                print(i,':',C_n[i])
            
            #calculate supports of each item in candidate set and prune
            for item in C_n:
                #if itemset is frequent, then subsets must also be frequent (and not in discarded)
                if item not in discarded:
                    sup_calc = C_n[item] / len(set_list)
                    if sup_calc >= min_support:
                        L_n[item] = C_n[item]
                    else:
                        discarded[item] = sup_calc
             
            #print passing set
            print('\nL{} set: \n'.format(n))
            for i in L_n:
                print(i,':',L_n[i])
            print('#'*75)
            
            if L_n == {} or L_n is None:
                stop = True
                return L_n_minus_1
                break
            n += 1
    return L_n_minus_1
            
def makeAssociationRules(inputset,min_confidence,transactions):
    print("Association Rules that meet minimum confidence: \n")
    for i in inputset:
        i = list(i)
        for j in i:
            right = {j}
            left = set(i) - right
            both = i
            counts_both = scanTransactions(both,transactions)
            counts_left = scanTransactions(list(left),transactions)
            sup_both = counts_both / len(transactions)
            sup_left = counts_left / len(transactions)
            if sup_left != 0:
                if sup_both / sup_left >= min_confidence:
                    print(left,'->',right,'    ','confidence: ',(sup_both/sup_left),'\n')                    
                    
                    
conn = create_connection(r"C:\Users\Nick\CS634_Data_Mining\auto_best_sellers_5.db")
transactionsList = getTransactions(conn,20)
results = apriori(transactionsList,0.5)
makeAssociationRules(results,.7,transactionsList)

2.6.0

Iteration 1
C1 candidate set: 

Cabin Filter with Activated Carbon : 14
Car Seat Covers : 14
Car Trash Can : 10
Car Vacuum : 20
Cleaning Gel : 10
Headrest Hanger : 14
RV Sewer Hose : 7
Windshield Sun Shade : 16
Wiper Blade : 6
Cleaning Putty : 11
Garbage Guard : 13
Heavy Duty Threadlocker : 4
Car Wax Polish Spray : 5
Diamond Steering Wheel Cover : 4
Gel Seat Cushion : 12
Interior Cleaner : 4
Microfiber Cleaning Cloth : 11
Window Shade : 6
10-Foot RV Sewer Hose Extension Kit : 6
Bike Phone Holder : 2
LED Car Strip Lights : 4
Rubber Floor Mats : 2
Trunk Organizers and Storage : 5
Office Chair Seat Cushion : 1
Portable Car Vacuum : 4
2-Piece Sun Shade : 2
RV Toilet Treatment : 4
Bumper Retainer Clips : 2

L1 set: 

Cabin Filter with Activated Carbon : 14
Car Seat Covers : 14
Car Trash Can : 10
Car Vacuum : 20
Cleaning Gel : 10
Headrest Hanger : 14
Windshield Sun Shade : 16
Cleaning Putty : 11
Garbage Guard : 13
Gel Seat Cushion : 12
Microfiber Cleaning Cloth : 11
##################

In [317]:
##brute force method

def bruteForce(set_list,min_support):
    n = 1
    C = {}
    L = {}
    stop = False
    while stop == False:
        if n <= 1:
            for transaction in range(len(set_list)):
                for item in set_list[transaction]:
                    #prevent duplicate scans of same subset
                    if type(item) != type(['type']):
                        sup_count = scanTransactions({item},set_list)
                        C[item] = sup_count
                    else:
                        sup_count = scanTransactions(item,set_list)
                        C[item] = sup_count

            #print first results
            print('#' * 50)
            print('\nIteration {} Set:\n'.format(n))

            for i in C:
                print(i,':',C[i])
            C_n = C
            n += 1
        else:
            
            #union of the sets
            sets = []
            C_n = list(C_n)
            sets = getUnions(C_n,n)
            print(sets)
            #save last iteration's results to a variable
            C_n_minus_1 = C_n
            
            #reinitialize L_n for the following iteration
            C_n = {}
            
            for i in range(len(sets)):
                support_count = scanTransactions(sets[i],set_list)
                C_n[tuple(sets[i])] = support_count
            
            #print set
            #print('#' * 50)
            #print('\nIteration {}'.format(n))
            #print('C{} candidate set: \n'.format(n))
            
            #list that will hold a value if the minimum support is reached
            #if no sets meet the minimum support, the loop will break
            passing_supports = {}
            for i in C_n:
                print(i,':',C_n[i])
                if C_n[i] / len(set_list) >= min_support:
                    passing_supports[i] = C_n[i]
            
            if passing_supports == {} or passing_supports is None:
                stop = True
                return C_n_minus_1
                break
            
            if C_n == {} or C_n is None:
                stop = True
                return C_n_minus_1
                break
            n += 1
            
def makeAssociationRulesBrute(inputset,min_support,min_confidence,transactions):
    print('#'*75)
    print("Association Rules that meet minimum confidence: \n")
    for i in inputset:
        i = list(i)
        counts_i = scanTransactions(i,transactions)
        sup_i = counts_i / len(transactions)
        if sup_i >= min_support:
            for j in i:
                counts_j = scanTransactions({j},transactions)
                sup_j = counts_j / len(transactions)
                right = {j}
                left = set(i) - right
                both = i
                counts_both = scanTransactions(both,transactions)
                counts_left = scanTransactions(list(left),transactions)
                counts_right = scanTransactions(list(right),transactions)
                sup_both = counts_both / len(transactions)
                sup_left = counts_left / len(transactions)
                sup_right = counts_right / len(transactions)
                if sup_left != 0:
                    if sup_left >= min_support and sup_right >= min_support:
                        if sup_both / sup_left >= min_confidence:
                            print(left,'->',right,'    ','confidence: ',(sup_both/sup_left),'\n')    


brute_result = bruteForce(transactionsList,.5)
makeAssociationRulesBrute(brute_result,.5,.7,transactionsList)

##################################################

Iteration 1 Set:

Cabin Filter with Activated Carbon : 15
Car Trash Can : 10
Cleaning Gel : 10
Cleaning Putty : 12
Gel Seat Cushion : 13
RV Sewer Hose : 7
RV Toilet Treatment : 18
Heavy Duty Threadlocker : 4
Windshield Sun Shade : 12
Car Wax Polish Spray : 5
Diamond Steering Wheel Cover : 4
Garbage Guard : 6
Headrest Hanger : 6
Interior Cleaner : 4
Microfiber Cleaning Cloth : 11
Window Shade : 6
Wiper Blade : 5
10-Foot RV Sewer Hose Extension Kit : 6
Bike Phone Holder : 2
Car Seat Covers : 6
LED Car Strip Lights : 4
Rubber Floor Mats : 2
Trunk Organizers and Storage : 5
Office Chair Seat Cushion : 1
Portable Car Vacuum : 4
2-Piece Sun Shade : 2
Bumper Retainer Clips : 2
Car Vacuum : 3
[{'Car Trash Can', 'Cabin Filter with Activated Carbon'}, {'Cleaning Gel', 'Cabin Filter with Activated Carbon'}, {'Cleaning Putty', 'Cabin Filter with Activated Carbon'}, {'Gel Seat Cushion', 'Cabin Filter with Activated Carbon'}, {'RV Sewer Hose', 'Cab

[{'Cleaning Gel', 'Car Trash Can', 'Cabin Filter with Activated Carbon'}, {'Car Trash Can', 'Cleaning Putty', 'Cabin Filter with Activated Carbon'}, {'Car Trash Can', 'Gel Seat Cushion', 'Cabin Filter with Activated Carbon'}, {'Car Trash Can', 'RV Sewer Hose', 'Cabin Filter with Activated Carbon'}, {'Car Trash Can', 'RV Toilet Treatment', 'Cabin Filter with Activated Carbon'}, {'Heavy Duty Threadlocker', 'Car Trash Can', 'Cabin Filter with Activated Carbon'}, {'Car Trash Can', 'Windshield Sun Shade', 'Cabin Filter with Activated Carbon'}, {'Car Trash Can', 'Car Wax Polish Spray', 'Cabin Filter with Activated Carbon'}, {'Car Trash Can', 'Cabin Filter with Activated Carbon', 'Diamond Steering Wheel Cover'}, {'Car Trash Can', 'Garbage Guard', 'Cabin Filter with Activated Carbon'}, {'Car Trash Can', 'Headrest Hanger', 'Cabin Filter with Activated Carbon'}, {'Car Trash Can', 'Interior Cleaner', 'Cabin Filter with Activated Carbon'}, {'Microfiber Cleaning Cloth', 'Car Trash Can', 'Cabin Filt

('Cleaning Gel', 'LED Car Strip Lights', 'Gel Seat Cushion') : 1
('Cleaning Gel', 'Rubber Floor Mats', 'Gel Seat Cushion') : 2
('Cleaning Gel', 'Gel Seat Cushion', 'Trunk Organizers and Storage') : 1
('Cleaning Gel', 'Office Chair Seat Cushion', 'Gel Seat Cushion') : 0
('Cleaning Gel', 'Portable Car Vacuum', 'Gel Seat Cushion') : 1
('Cleaning Gel', 'Gel Seat Cushion', '2-Piece Sun Shade') : 2
('Cleaning Gel', 'Gel Seat Cushion', 'Bumper Retainer Clips') : 1
('Cleaning Gel', 'Car Vacuum', 'Gel Seat Cushion') : 1
('Cleaning Gel', 'RV Toilet Treatment', 'RV Sewer Hose') : 5
('Cleaning Gel', 'Heavy Duty Threadlocker', 'RV Sewer Hose') : 0
('Cleaning Gel', 'Windshield Sun Shade', 'RV Sewer Hose') : 3
('Cleaning Gel', 'RV Sewer Hose', 'Car Wax Polish Spray') : 2
('Cleaning Gel', 'RV Sewer Hose', 'Diamond Steering Wheel Cover') : 2
('Cleaning Gel', 'Garbage Guard', 'RV Sewer Hose') : 2
('Cleaning Gel', 'Headrest Hanger', 'RV Sewer Hose') : 2
('Cleaning Gel', 'Interior Cleaner', 'RV Sewer Hose

('Portable Car Vacuum', 'Gel Seat Cushion', 'Garbage Guard') : 2
('Garbage Guard', 'Gel Seat Cushion', '2-Piece Sun Shade') : 1
('Gel Seat Cushion', 'Garbage Guard', 'Bumper Retainer Clips') : 1
('Car Vacuum', 'Garbage Guard', 'Gel Seat Cushion') : 1
('Headrest Hanger', 'Interior Cleaner', 'Gel Seat Cushion') : 3
('Microfiber Cleaning Cloth', 'Headrest Hanger', 'Gel Seat Cushion') : 1
('Headrest Hanger', 'Window Shade', 'Gel Seat Cushion') : 3
('Gel Seat Cushion', 'Headrest Hanger', 'Wiper Blade') : 2
('Headrest Hanger', 'Gel Seat Cushion', '10-Foot RV Sewer Hose Extension Kit') : 1
('Headrest Hanger', 'Gel Seat Cushion', 'Bike Phone Holder') : 1
('Headrest Hanger', 'Car Seat Covers', 'Gel Seat Cushion') : 1
('Headrest Hanger', 'LED Car Strip Lights', 'Gel Seat Cushion') : 1
('Headrest Hanger', 'Rubber Floor Mats', 'Gel Seat Cushion') : 0
('Headrest Hanger', 'Gel Seat Cushion', 'Trunk Organizers and Storage') : 1
('Headrest Hanger', 'Office Chair Seat Cushion', 'Gel Seat Cushion') : 1


('Heavy Duty Threadlocker', 'Bike Phone Holder', 'Microfiber Cleaning Cloth') : 0
('Heavy Duty Threadlocker', 'Car Seat Covers', 'Microfiber Cleaning Cloth') : 0
('Heavy Duty Threadlocker', 'LED Car Strip Lights', 'Microfiber Cleaning Cloth') : 0
('Heavy Duty Threadlocker', 'Rubber Floor Mats', 'Microfiber Cleaning Cloth') : 0
('Heavy Duty Threadlocker', 'Trunk Organizers and Storage', 'Microfiber Cleaning Cloth') : 0
('Heavy Duty Threadlocker', 'Office Chair Seat Cushion', 'Microfiber Cleaning Cloth') : 0
('Heavy Duty Threadlocker', 'Portable Car Vacuum', 'Microfiber Cleaning Cloth') : 1
('Heavy Duty Threadlocker', '2-Piece Sun Shade', 'Microfiber Cleaning Cloth') : 0
('Heavy Duty Threadlocker', 'Bumper Retainer Clips', 'Microfiber Cleaning Cloth') : 0
('Heavy Duty Threadlocker', 'Car Vacuum', 'Microfiber Cleaning Cloth') : 0
('Heavy Duty Threadlocker', 'Window Shade', 'Wiper Blade') : 0
('Heavy Duty Threadlocker', 'Window Shade', '10-Foot RV Sewer Hose Extension Kit') : 1
('Heavy Dut

('Headrest Hanger', 'Office Chair Seat Cushion', 'LED Car Strip Lights') : 1
('Headrest Hanger', 'Portable Car Vacuum', 'LED Car Strip Lights') : 1
('Headrest Hanger', 'LED Car Strip Lights', '2-Piece Sun Shade') : 0
('Headrest Hanger', 'LED Car Strip Lights', 'Bumper Retainer Clips') : 0
('Car Vacuum', 'Headrest Hanger', 'LED Car Strip Lights') : 0
('Headrest Hanger', 'Rubber Floor Mats', 'Trunk Organizers and Storage') : 0
('Headrest Hanger', 'Office Chair Seat Cushion', 'Rubber Floor Mats') : 0
('Headrest Hanger', 'Portable Car Vacuum', 'Rubber Floor Mats') : 0
('Headrest Hanger', 'Rubber Floor Mats', '2-Piece Sun Shade') : 0
('Headrest Hanger', 'Rubber Floor Mats', 'Bumper Retainer Clips') : 0
('Car Vacuum', 'Headrest Hanger', 'Rubber Floor Mats') : 0
('Headrest Hanger', 'Office Chair Seat Cushion', 'Trunk Organizers and Storage') : 1
('Headrest Hanger', 'Portable Car Vacuum', 'Trunk Organizers and Storage') : 1
('Headrest Hanger', 'Trunk Organizers and Storage', '2-Piece Sun Shade